In [108]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import time
import timeit
import csv
import os
import string
import textblob
import requests
import io
import nltk

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.decomposition import PCA
from sklearn.decomposition import RandomizedPCA

from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_curve, auc
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import preprocessing
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from importlib import reload
# reload(text_processing)


# nlp = spacy.load('en') # loading the language model 
#data = pd.read_feather('data/preprocessed_data') # reading a pandas dataframe which is stored as a feather file


%matplotlib inline
# sns.set_context('poster')
# sns.set_color_codes()


In [109]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
elapsed = timeit.default_timer() - start_time

### Data
- inspect data (ex: df.head(), df.groupby('labels').count())
- select

In [7]:
df_climate = pd.read_pickle("./Data/climate.pkl")
df_spam = pd.read_pickle("./Data/spam.pkl")
df_newsgroups = pd.read_pickle("./Data/newsgroups.pkl")
df_reuters = pd.read_pickle("./Data/reuters.pkl")
df_bbc = pd.read_pickle("./Data/bbc.pkl")

In [110]:
# df = pd.read_pickle("./Data/spam.pkl")
df = pd.read_pickle("./Data/newsgroups.pkl")

In [111]:
# unhash to inspect label count
# df.groupby('labels')['texts'].count()

In [112]:
# split dataframe into text and labels
texts = df.texts
labels = df.labels
processed_texts = df.processed_text

In [113]:
# UNPROCESSED: raw text, split the dataset into training and validation datasets 
train_x, valid_x, train_y_text_label, valid_y_text_label = train_test_split(texts, labels)

# PROCESSED: preprocessed text, split the dataset into training and validation datasets 
train_xp, valid_xp, train_yp_text_label, valid_yp_text_label = train_test_split(processed_texts, labels)

# label encode the target variable for raw and processed datasets
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y_text_label)
valid_y = encoder.fit_transform(valid_y_text_label)
train_yp = encoder.fit_transform(train_yp_text_label)
valid_yp = encoder.fit_transform(valid_yp_text_label)

### Binarize data labels
- for binary classification problem --> takes one category (ex: label '1') and makes '1' all others '0'

In [114]:
train_y = np.array((pd.DataFrame({'t_label':train_y}).t_label == 1).astype('int'))
valid_y = np.array((pd.DataFrame({'t_label':valid_y}).t_label == 1).astype('int'))
train_yp = np.array((pd.DataFrame({'t_label':train_yp}).t_label == 1).astype('int'))
valid_yp = np.array((pd.DataFrame({'t_label':valid_yp}).t_label == 1).astype('int'))

### Test on merged climate data

In [115]:
def climate_merge_df(other_df):
    '''
    Args: df_other: a pd.dataframe with columns: 'texts','labels','processed_text'
    Returns: a df with climate and random sample of other data 50/50 split
    Note: uncomment other labels
    '''
    
    df = pd.read_pickle("./Data/climate.pkl")
    length = min(df.shape[0],other_df.shape[0])
    other_s = other_df.sample(length)
    # df_other_s.labels = 0
    df_s = df.sample(length)
    other_s.append(df_s)
    
    return other_s.append(df_s)

## Feature Engineering

- https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

### Count Vectors

In [116]:
# params take form of (analyzer, token pattern, stop_words)

def run_vectorizer(train_x,valid_x, model, params):

    # create a vectorizer object 
    if model == CountVectorizer:
        vect = model(analyzer=params[0], token_pattern=params[1], stop_words=params[2], 
                     lowercase=params[3], ngram_range=params[4])
    
    else:
        vect = model(model(analyzer=params[0], max_df=params[1], min_df=params[2], max_features=params[3], 
          stop_words=params[4], use_idf=params[5]), token_pattern=params[6], ngram_range=params[7])
    
    
    vect.fit(texts)

    # transform the training and validation data using count vectorizer objectea
    xtrain_ =  vect.transform(train_x)
    xvalid_ =  vect.transform(valid_x)
    
    return xtrain_, xvalid_

In [117]:
'''CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)[source]¶
'''

'CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=’(?u)\x08\\w\\w+\x08’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)[source]¶\n'

In [118]:
# count vectorizer params
model_count = CountVectorizer

params_count1 = ('word', r'\w{1,}', 'english', True, (1,1))
params_count2 = ('char', r'\w{1,}', 'english', True, (1,1))
params_count3 = ('word', r'\w{1,}', 'english', True, (2,3))
params_count4 = ('char', r'\w{1,}', 'english', True, (2,3))

xtrain_count,xvalid_count= run_vectorizer(train_x,valid_x, model_count, params_count1)
xtrain_count_char,xvalid_count_char = run_vectorizer(train_x,valid_x, model_count, params_count2)
xtrain_count_ngram,xvalid_count_ngram = run_vectorizer(train_x,valid_x, model_count, params_count3)
xtrain_count_ngram_chars,xvalid_count_ngram_chars = run_vectorizer(train_x,valid_x, model_count, params_count4)

MemoryError: 

### TF-IDF Vectors

#### Tfidf vectorizer:
- Strips out “english" stop words
- Filters out terms that occur in more than half of the docs (max_df=0.5)
- Filters out terms that occur in only one document (min_df=2).
- Selects the 10,000 most frequently occuring words in the corpus.
- Normalizes the vector (L2 norm of 1.0) to normalize the effect of document length on the tf-idf values. 

In [ ]:
# tfidf vectorizer params
model_tfidf = TfidfVectorizer

params_tfidf1 = ('word',0.5,2,10000,'english',True, r'\w{1,}', (1,1))
params_tfidf2 = ('char',0.5,2,10000,'english',True, r'\w{1,}', (1,1))
params_tfidf3 = ('word',0.5,2,10000,'english',True, r'\w{1,}', (2,3))
params_tfidf4 = ('char',0.5,2,10000,'english',True, r'\w{1,}', (2,3))

xtrain_tfidf,xvalid_tfidf= run_vectorizer(train_x,valid_x, model_tfidf, params_tfidf1)
xtrain_tfidf_char,xvalid_tfidf_char = run_vectorizer(train_x,valid_x, model_tfidf, params_tfidf2)
xtrain_tfidf_ngram,xvalid_tfidf_ngram = run_vectorizer(train_x,valid_x, model_tfidf, params_tfidf3)
xtrain_tfidf_ngram_chars,xvalid_tfidf_ngram_chars = run_vectorizer(train_x,valid_x, model_tfidf, params_tfidf4)

In [ ]:
%timeit run_vectorizer(train_xp,valid_xp, model_tfidf, params_tfidf1)

### Text / NLP based features

In [ ]:
def get_NLP_features(texts, normed_=True):

    trainDF=pd.DataFrame()
    trainDF['char_count'] = texts.apply(len)
    trainDF['word_count'] = texts.apply(lambda x: len(x.split()))
    trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
    trainDF['punctuation_count'] = texts.apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
    trainDF['title_word_count'] = texts.apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
    trainDF['upper_case_word_count'] = texts.apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
    
    pos_family = {
        'noun' : ['NN','NNS','NNP','NNPS'],
        'pron' : ['PRP','PRP$','WP','WP$'],
        'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
        'adj' :  ['JJ','JJR','JJS'],
        'adv' : ['RB','RBR','RBS','WRB']
    }
    
    # function to check and get the part of speech tag count of a words in a given sentence
    def check_pos_tag(x, flag):
        cnt = 0
        try:
            wiki = textblob.TextBlob(x)
            for tup in wiki.tags:
                ppo = list(tup)[1]
                if ppo in pos_family[flag]:
                    cnt += 1
        except:
            pass
        return cnt
    
    trainDF['noun_count'] = texts.apply(lambda x: check_pos_tag(x, 'noun'))
    trainDF['verb_count'] = texts.apply(lambda x: check_pos_tag(x, 'verb'))
    trainDF['adj_count'] = texts.apply(lambda x: check_pos_tag(x, 'adj'))
    trainDF['adv_count'] = texts.apply(lambda x: check_pos_tag(x, 'adv'))
    trainDF['pron_count'] = texts.apply(lambda x: check_pos_tag(x, 'pron'))
    
    if normed_:
        
        x = trainDF.values #returns a numpy array
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        trainDF_normed = pd.DataFrame(x_scaled)
        
        return trainDF_normed
    
    return trainDF 

In [ ]:
NLP_DF = get_NLP_features(train_x)

### Topic Model

-  https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [ ]:
def list_dictionary(dictionary, n=10):
    '''
    Returns: the first n words from dictionary 
    '''
    
    count = 0
    for k, v in dictionary.iteritems():
        print(k, v)
        count += 1
        if count > n:
            break
            
# list_dictionary(dictionary, top=10)

In [ ]:
def inspect_corpus(corpus,doc_to_insp=10):
    doc_insp = corpus[doc_to_insp]
    for i in range(len(doc_insp)):
        print("Word {} (\"{}\") appears {} time.".format(doc_insp[i][0],dictionary[doc_insp[i][0]],doc_insp[i][1]))

In [ ]:
# print topics from lda_model
def get_topics(model):
    for idx, topic in model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
# get_topics(lda_model)

In [ ]:
# inspect where doc would be classified
def lda_in_doc_classification(model, corpus, doc_num=100):
    for index, score in sorted(model[corpus[doc_num]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))
# lda_in_doc_classification(lda_model, bow_corpus, 20)
# lda_in_doc_classification(lda_model_tfidf, bow_corpus, 100)

In [ ]:
def lda_out_doc_classification(unseen_document):
    '''
    Takes a list of a single doc string
    example: unseen_document = ['The night love tomorrow']
    '''
    text = pre_processing_steps(pd.DataFrame({'texts': unseen_document})).processed_text[0]
    bow_vector = dictionary.doc2bow(text)
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
# build dictionary, bow corpus, and tfidf corpus (using bow corpus)
dictionary = corpora.Dictionary(train_xp)
dictionary.filter_extremes(no_below=20, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in train_xp]
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
# Running LDA using Bag of Words
lda_model = models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
# Running LDA using TF-IDF
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
# get_topics(lda_model_tfidf)

In [ ]:
# lda_model_tfidf.show_topics(10)
# lda_model_tfidf.get_topics().shape

#### LDA TFIDF similarity

In [ ]:
# TFIDF
# https://www.oreilly.com/learning/how-do-i-compare-document-similarity-using-python
TFIDF = models.TfidfModel(corpus)
sims = similarities.Similarity('temp' , TFIDF[corpus], num_features = len(dictionary))
q_doc = dictionary.doc2bow(text) # prediction similarity --> query doc
q_doc_TFIDF = TFIDF[q_doc]
temp_lst=[]
for idx,i in enumerate(sims[q_doc_TFIDF]):
    if i != 0.:
        temp_lst.append((idx,i))
        
# inspect_corpus(corpus,1704)

## Models

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return classification_report(predictions,valid_y)

In [ ]:
def plot_confusion_matrix(valid_y, y_predicted):
    cm = confusion_matrix(valid_y, y_predicted)
    df_cm = pd.DataFrame(cm)
    fig = plt.figure(figsize=(4,3))
    heatmap = sns.heatmap(df_cm,annot=True,cmap='Blues', fmt='g', cbar=False)

### NB

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

accuracy = train_model(MultinomialNB(), xtrain_count_char, train_y, xvalid_count_char)
print("NB, Count Vectors Char: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

### KNN


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

accuracy = train_model(KNeighborsClassifier(), xtrain_count, train_y, xvalid_count)
print("KNN, Count Vectors: ", accuracy)

accuracy = train_model(KNeighborsClassifier(), xtrain_count_char, train_y, xvalid_count_char)
print("KNN, Count Vectors Char: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("KNN, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("KNN, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("KNN, CharLevel Vectors: ", accuracy)

### SVM

In [ ]:
# SVM N-Gram Chars on TF IDF Vectors
from sklearn import svm
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("SVM, N-Gram Chars on TF IDF Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

#### SVM experiments

In [ ]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),])
_ = text_clf_svm.fit(train_x, train_y)
predicted_svm = text_clf_svm.predict(valid_x)
np.mean(predicted_svm == valid_y)

#### SVM grid search cv

In [ ]:
from sklearn.svm import LinearSVC

pipeline = Pipeline([
        ('vect', TfidfVectorizer(min_df=3, max_df=0.95)),
        ('clf', LinearSVC(C=1000)),
    ])

# TASK: Build a grid search to find out whether unigrams or bigrams are
# more useful.
# Fit the pipeline on the training set using grid search for the parameters
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
}
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=kfolds,verbose=1)
grid_search.fit(train_x, train_y)

# TASK: print the mean and std for each candidate along with the parameter
# settings for all the candidates explored by grid search.
n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f'
             % (grid_search.cv_results_['params'][i],
                grid_search.cv_results_['mean_test_score'][i],
                grid_search.cv_results_['std_test_score'][i]))

y_predicted = grid_search.predict(valid_x)

# Print the classification report
print(classification_report(valid_y, y_predicted,))
                                    
# Print and plot the confusion matrix
plot_confusion_matrix(valid_y, y_predicted)

In [ ]:
# en_stopwords = set(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenizer,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = stopwords)

kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="linear", class_weight="balanced"))


grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm.fit(train_x, train_y)
grid_svm.score(valid_x, valid_y)


predictions_svm_ = grid_svm.predict(valid_x)
cm = confusion_matrix(valid_y, predictions_svm_)
print(classification_report(valid_y,predictions_svm_))
plot_confusion_matrix(valid_y, predictions_svm_)

In [ ]:
# inspect errors
# error_df = pd.DataFrame({'val_text':valid_x, 'val_true':valid_y, 'val_pred':predictions_svm_})
# error_df[(error_df['val_pred']!=error_df['val_true'])].sample(10)

### Logistic Regression

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

### Random Forest

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

### Word Embeddings

In [ ]:
from keras.preprocessing import text, sequence

# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('./Data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer in keras
token = text.Tokenizer()
token.fit_on_texts(texts)
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Keras

In [ ]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

### NN

In [ ]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print("NN, Ngram Level TF IDF Vectors",  accuracy)

###  CNN

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)



### LSTM

In [ ]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

### Bidirectional RNN

In [ ]:
def create_bidirectional_rnn():
# Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

### RCNN

In [ ]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

## Tensorflow - Logistic Regression

In [ ]:
# Start a graph session
sess = tf.Session()

batch_size = 200
max_features = 1000

In [ ]:
# Split up data set into train/test
train_indices = np.random.choice(sparse_tfidf_texts.shape[0], round(0.8*sparse_tfidf_texts.shape[0]), replace=False)
test_indices = np.array(list(set(range(sparse_tfidf_texts.shape[0])) - set(train_indices)))
texts_train = sparse_tfidf_texts[train_indices]
texts_test = sparse_tfidf_texts[test_indices]
target_train = np.array([x for ix, x in enumerate(labels) if ix in train_indices])
target_test = np.array([x for ix, x in enumerate(labels) if ix in test_indices])

In [ ]:
# Create variables for logistic regression
A = tf.Variable(tf.random_normal(shape=[max_features,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Initialize placeholders
x_data = tf.placeholder(shape=[None, max_features], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Declare logistic model (sigmoid in loss function)
model_output = tf.add(tf.matmul(x_data, A), b)

# Declare loss function (Cross Entropy loss)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_output, labels=y_target))

# Actual Prediction
prediction = tf.round(tf.sigmoid(model_output))
predictions_correct = tf.cast(tf.equal(prediction, y_target), tf.float32)
accuracy = tf.reduce_mean(predictions_correct)

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.0025)
train_step = my_opt.minimize(loss)

# Intitialize Variables
init = tf.global_variables_initializer()
sess.run(init)

# Start Logistic Regression
train_loss = []
test_loss = []
train_acc = []
test_acc = []
i_data = []
for i in range(10000):
    rand_index = np.random.choice(texts_train.shape[0], size=batch_size)
    rand_x = texts_train[rand_index].todense()
    rand_y = np.transpose([target_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    
    # Only record loss and accuracy every 100 generations
    if (i+1)%100==0:
        i_data.append(i+1)
        train_loss_temp = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
        train_loss.append(train_loss_temp)
        
        test_loss_temp = sess.run(loss, feed_dict={x_data: texts_test.todense(), y_target: np.transpose([target_test])})
        test_loss.append(test_loss_temp)
        
        train_acc_temp = sess.run(accuracy, feed_dict={x_data: rand_x, y_target: rand_y})
        train_acc.append(train_acc_temp)
    
        test_acc_temp = sess.run(accuracy, feed_dict={x_data: texts_test.todense(), y_target: np.transpose([target_test])})
        test_acc.append(test_acc_temp)
    if (i+1)%500==0:
        acc_and_loss = [i+1, train_loss_temp, test_loss_temp, train_acc_temp, test_acc_temp]
        acc_and_loss = [np.round(x,2) for x in acc_and_loss]
        print('Generation # {}. Train Loss (Test Loss): {:.2f} ({:.2f}). Train Acc (Test Acc): {:.2f} ({:.2f})'.format(*acc_and_loss))


# Plot loss over time
plt.plot(i_data, train_loss, 'k-', label='Train Loss')
plt.plot(i_data, test_loss, 'r--', label='Test Loss', linewidth=4)
plt.title('Cross Entropy Loss per Generation')
plt.xlabel('Generation')
plt.ylabel('Cross Entropy Loss')
plt.legend(loc='upper right')
plt.show()

# Plot train and test accuracy
plt.plot(i_data, train_acc, 'k-', label='Train Set Accuracy')
plt.plot(i_data, test_acc, 'r--', label='Test Set Accuracy', linewidth=4)
plt.title('Train and Test Accuracy')
plt.xlabel('Generation')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

## COMPARISON
- https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/

In [ ]:
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# load dataset
'''
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = train_y
'''
X = NLP_DF
Y = train_y
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()